In [36]:
#load data
import pandas as pd
def load_data(csv_file, sample_fraction=1.0):
    """Loads Question Pairs from a CSV file

    Args:
        csv_file (str): Path to csv_file
        sample_fraction (float): Fraction of data to sample, default is 1.0
    Returns:
        tuple: A tuple containing supervised data pairs
        returns [],[] on error
    """
    sentences1 = []
    sentences2 = []
    is_duplicate = []
    
    try:
        df = pd.read_csv(csv_file, encoding='utf-8')
        #print("Column names:", df.columns)
        
        if sample_fraction < 1.0:
            df = df.sample(frac=sample_fraction, random_state=42).reset_index(drop=True)
        
        sentence1_series = df['question1']
        sentence2_series = df['question2']
        is_duplicate_series = df['is_duplicate']
        
        sentences1 = sentence1_series.tolist()
        sentences2 = sentence2_series.tolist()
        is_duplicate = is_duplicate_series.tolist()
        
        if len(sentences1) != len(sentences2):
            raise ValueError("The number of sentences in question1 and question2 do not match.")
        else:
            print(f"Loaded {len(sentences1)} sentences.")
        return sentences1, sentences2, is_duplicate
        
    except FileNotFoundError:
        print("Wrong Path")
        return [],[],[]
    
    except Exception as e:
        print(f"An {e} Error Occurred")
        return [],[],[]

DATA_PATH = r'C:/Users/Jash\Documents/Research\Semantic Equivilance\SemanticEquivilance/question_pairs/questions.csv'
sentences1, sentences2, value = load_data(DATA_PATH, sample_fraction=0.001)
data_pairs = list(zip(sentences1, sentences2, value))


Loaded 404 sentences.


In [37]:
import pennylane as qml
import numpy as np
from lambeq import AtomicType, BobcatParser, StronglyEntanglingAnsatz, Rewriter
from lambeq.backend.pennylane import to_pennylane as to_qml
import random

def swap_test(state1_vec, state2_vec, num_qubits, initial_state=0):
    """
    Performs a Quantum Swap Test between two quantum state vectors.

    Args:
        state1_vec (np.ndarray): The first state vector.
        state2_vec (np.ndarray): The second state vector.

    Returns:
        float: The estimated squared overlap (fidelity) between the two states.
    """
    if 2**num_qubits != len(state1_vec):
        raise ValueError("State vectors must have a length that is a power of 2.")

    total_qubits = 1 + 2 * num_qubits #1 Ancilla qubit + 2 state qubits

    dev = qml.device("default.qubit", wires=total_qubits, shots=10000) # N = 10000 runs of the circuit for statistical significance

    @qml.qnode(dev)
    def circuit(): #|0 , psi, phi>
        # Step 1: Prepare the ancilla qubit in a superposition
        qml.Hadamard(wires=0)
        print(f"Comparing: {state1_vec} and {state2_vec}")
        # Step 2: Prepare the two input states
        #basis for protocol 1
        qml.StatePrep(state1_vec, wires=range(1, 1 + num_qubits), normalize=True)
        #basis for protocol 1
        qml.StatePrep(state2_vec, wires=range(1 + num_qubits, 1 + 2 * num_qubits))

        # Step 3: Apply controlled-SWAP gates
        for i in range(num_qubits):
            qml.CSWAP(wires=[0, 1 + i, 1 + num_qubits + i]) #selects every register of phi and psi for swap

        # Step 4: Apply Hadamard to ancilla
        qml.Hadamard(wires=0)
        # Step 5: Measure the ancilla qubit
        return qml.sample(wires=0)

    measurement_results = circuit()
    squared_overlap = 1 - 2/len(measurement_results) * np.sum(measurement_results)
    
    return abs(squared_overlap)

In [38]:
def parameter_shift(qnode, structured_params, param_index, shift=np.pi/2):
    """
    Calculates the gradient of a QNode with respect to a single parameter
    using the parameter-shift rule.

    Args:
        qnode (qml.QNode): The quantum circuit QNode.
        structured_params (list): The list of structured parameters.
        param_index (tuple): A tuple (group_index, element_index) identifying
                             the parameter to shift.
        shift (float): The shift value, typically pi/2.

    Returns:
        float: The calculated gradient.
    """
    group_idx, elem_idx = param_index

    # Create a copy of the parameters to avoid modifying the original list
    shifted_params_plus = structured_params.copy()
    shifted_params_minus = structured_params.copy()
    
    # Apply the positive and negative shifts
    shifted_params_plus[group_idx][elem_idx] += shift
    shifted_params_minus[group_idx][elem_idx] -= shift
    
    # Calculate the circuit output for both shifted parameters
    plus_result = qnode(shifted_params_plus)
    minus_result = qnode(shifted_params_minus)
    
    # Apply the parameter-shift rule formula
    gradient = (plus_result - minus_result) / (2 * np.sin(shift))
    
    return gradient

In [52]:
def lambeq_sentence_to_circuit(sentence, ansatz, parser, rewriter, return_type='state', include_debug_prints=False, expval = qml.PauliZ(0)):
    """
    Converts a natural language sentence into a quantum state vector
    using Lambeq's BobcatParser and IQPAnsatz, handling parameterization
    via PennyLaneModel.

    Args:
        sentence (str): The input sentence.
        ansatz (lambeq.ansatz.Ansatz): The quantum ansatz to apply.
        parser (lambeq.parser.Parser): The parser to convert sentence to diagram. This is a GPT
        rewriter (lambeq.rewrite.Rewriter): The rewriter to simplify the diagram. This is a GPT
        include_debug_prints (bool): Whether to include detailed debug prints.
        start_basis_state (int): The basis state to initialize the qubits. Default is |0>.
        seed (int, optional): Seed for random number generation for reproducibility.
        return_type (str): 'state' or 'expval' to specify the return type.
        expval (qml.Observable): The observable for expval value calculation.

    Returns:
        tuple: A tuple containing the state vector (np.ndarray) and
               the number of qubits (int).
    """
    try:
        if include_debug_prints:
            print(f"\n--- Debugging: Sentence '{sentence}' ---")

        # Step 1: Convert sentence to a DisCoPy diagram
        diagram = parser.sentence2diagram(sentence)
        if include_debug_prints:
            print("Step 1: Sentence parsed to diagram.")

        # Step 2: Rewrite the diagram
        rewritten_diagram = rewriter(diagram)
        if include_debug_prints:
            print("Step 2: Diagram rewritten.")

        # Step 3: Normalize the diagram
        normalized_diagram = rewritten_diagram.normal_form()
        if include_debug_prints:
            print("Step 3: Diagram normalized.")
            
        # Step 4: Apply the ansatz to the normalized diagram to get a DisCoPy circuit
        circuit = ansatz(normalized_diagram)
        if include_debug_prints:
            print("Step 4: Ansatz applied to create DisCoPy circuit.")

        # Step 5: Convert the DisCoPy circuit to a PennyLane circuit object
        temp_qml_circuit = to_qml(circuit)
        num_qubits = temp_qml_circuit._n_qubits
        
        if include_debug_prints:
            print("Step 5: DisCoPy circuit converted to PennyLane object.")
            
        param_structure = temp_qml_circuit._params
        
        if include_debug_prints:
            print(f"Parameter structure: {param_structure}")
            print(f"Number of qubits: {num_qubits}, Parameter groups: {len(param_structure)}")
        
        # Build parameters in the exact same structure as _params
        structured_params = []
        
        for param_group in param_structure:
            if isinstance(param_group, list) and len(param_group) > 0:
                # This parameter group has parameters - create random values for each
                group_values = [random.uniform(0.1, 2 * np.pi - 0.1) for _ in param_group]
                structured_params.append(group_values)
            else:
                # Empty parameter group
                structured_params.append([])
            
        if include_debug_prints:
            print("Step 6: Structured parameters generated.")
            print(f"Structured params lengths: {[len(group) for group in structured_params]}")
        
        dev = qml.device("default.qubit", wires=num_qubits, shots=10000) # N = 10000 runs of the circuit for statistical significance
        
        @qml.qnode(dev)
        def qnode_circuit(structured_params):
            circuit_func = temp_qml_circuit.make_circuit()
            circuit_func(structured_params)
            if return_type == 'expval':
                return qml.expval(expval)
            else:
                return qml.state()  # Return the state vector after applying the circuit
        
        return qnode_circuit, param_structure, num_qubits

    except Exception as e:
        print(f"CRITICAL ERROR: Failed to process circuit: {e}")
        import traceback
        traceback.print_exc()
        return None, None

In [40]:
def initialize_structured_params(param_structure, sentence_seed=None):
    """
    Initializes structured parameters based on the given parameter structure.

    Args:
        param_structure (list): The structure of parameters as a list of lists.
        seed (int): Seed for random number generation for reproducibility.

    Returns:
        list: A list of structured parameters with random values.
    """
    if sentence_seed is not None:
        random.seed(sentence_seed)
    structured_params = []
    for param_group in param_structure:
        if isinstance(param_group, list) and len(param_group) > 0:
            # This parameter group has parameters - create random values for each
            group_values = [random.uniform(0.1, 2 * np.pi - 0.1) for _ in param_group]
            structured_params.append(group_values)
        else:
            # Empty parameter group
            structured_params.append([])
    
    return structured_params

In [41]:
import math
def FischerInformation(Fidelity): #Statisitcal Information and DISTANCE metric Fubini-Study Metric/ Wooters Distance
    rootFidelity = math.sqrt(Fidelity) #Square root fidelity term
    return math.acos(rootFidelity)

In [ ]:
if __name__ == "__main__":
    parser = BobcatParser() 
    rewriter = Rewriter(['curry', 'prepositional_phrase', 'determiner'])
    # Increase ansatz parameters to get more complex quantum states
    ansatz = StronglyEntanglingAnsatz(
    {AtomicType.NOUN: 2, AtomicType.SENTENCE: 1}, 
    n_layers=1
)

    print("--- Generating states from sentences ---")

    sentence1 = "Alice loves the dog that Bob purchased."
    sentence2 = "Bob loves the dog that Alice sold."
    sentence3 = "The big cat sleeps peacefully."
    sentence4 = "The small bird sings loudly."
    sentence5 = "The lizard basks in the sun."
    sentence6 = "The sun shines on the lizard"
    
    sentences = [sentence1, sentence2, sentence3, sentence4, sentence5, sentence6]
    state_data = {}
    for s_idx, sentence in enumerate(sentences):
        try:
            state_vec, num_qubits = lambeq_sentence_to_circuit(sentence, ansatz, parser, rewriter, include_debug_prints=True)
            #state_vec, num_qubits = lambeq_sentence_to_state_vector(sentence, ansatz, parser, rewriter, include_debug_prints=False)
            state_data[sentence] = (state_vec, num_qubits)
            print(f"Sentence {s_idx+1}: '{sentence}'")
            print(f"Generated state with {num_qubits} qubits")
        except Exception as e:
            print(f"Error processing sentence '{sentence}': {e}")
            state_data[sentence] = (None, None)

    print("\n--- Performing Swap Tests ---")
    
    # Filter out sentences that did not produce valid states
    valid_sentences = [s for s in sentences if state_data[s][0] is not None and state_data[s][1] is not None and state_data[s][1] > 0]

    if not valid_sentences:
        print("No valid multi-qubit states generated. Cannot perform Swap Tests meaningfully.")
    else:
        first_num_qubits = state_data[valid_sentences[0]][1]
        all_same_qubits = all(state_data[s][1] == first_num_qubits for s in valid_sentences)

        if not all_same_qubits:
            print("\nWarning: Not all valid sentences resulted in circuits with the same number of qubits.")
            print("Swap Test requires states to have the same number of qubits.")
            print("Pairs with different qubit counts will be skipped.")
            for s in valid_sentences:
                print(f"  '{s}': {state_data[s][1]} qubits")

        for i in range(len(valid_sentences)):
            for j in range(i, len(valid_sentences)):
                s1 = valid_sentences[i]
                s2 = valid_sentences[j]

                vec1, nq1 = state_data[s1]
                vec2, nq2 = state_data[s2]

                if nq1 == nq2:
                    print(f"\nSwap Test between '{s1}' and '{s2}':")
                    # Fix: Use nq1 (or nq2, they're equal)
                    overlap = swap_test(vec1, vec2, nq1)
                    print(overlap)
                    Qangle = FischerInformation(overlap) # 0 - pi/2
                    NormalizedQangle = Qangle / (math.pi / 2) # 0 means identical, 1 means orthogonal
                    print(f"Estimated Quantum Angle: {NormalizedQangle:.4f}")
                    if s1 == s2:
                        print(" (Expected to be close to 1.0 for identical states)")
                else:
                    print(f"\nSkipping Swap Test between '{s1}' ({nq1} qubits) and '{s2}' ({nq2} qubits) due to different qubit counts.")

In [42]:
def loss_function(overlap, is_duplicate):
    """
    Revised loss function based on the overlap and a binary label.
    - For duplicates (is_duplicate=1), minimizes loss by driving overlap to 1.
    - For non-duplicates (is_duplicate=0), minimizes loss by reducing the overlap.
    """
    if is_duplicate == 1:
        # A duplicate pair should have an overlap of 1 (Quantum Angle = 0)
        return 1 - overlap
    else:
        # A non-duplicate pair should have a small overlap, but not necessarily 0.
        return overlap

In [53]:
def calculate_overlap(qnode_func, params1, params2):
    """
    Calculates the overlap between two quantum states prepared by the same QNode
    but with different parameters. This is a shortcut that only works with quantum simulators, for real quantum hardware, use the swap test function.

    Args:
        qnode_func (qml.QNode): The quantum circuit QNode.
        params1 (list): Structured parameters for the first state.
        params2 (list): Structured parameters for the second state.

    Returns:
        float: The estimated squared overlap (fidelity) between the two states.
    """
    # Get the state vectors for both sets of parameters
    state1 = qnode_func(params1)
    state2 = qnode_func(params2)
    
    # Calculate the overlap
    overlap = np.abs(np.vdot(state1, state2))**2
    
    return overlap

In [54]:
# Helper function to group sentences by qubit count
def calculate_pair_gradients(qnode_func, params, s1, s2, is_duplicate):
    """
    Calculates gradients for a pair of sentences with respect to the shared parameters.
    Returns a dictionary of gradients indexed by (group, element) tuple.
    """
    gradients = {idx: 0.0 for idx in [(g, e) for g in range(len(params)) for e in range(len(params[g]))]}

    # We need to compute the gradient for each parameter
    for group_idx, group in enumerate(params):
        for elem_idx in range(len(group)):
            param_index = (group_idx, elem_idx)

            # Shift the parameters for both sentences in the pair
            params_plus_s1 = params.copy()
            params_plus_s1[group_idx][elem_idx] += np.pi / 2
            
            params_minus_s1 = params.copy()
            params_minus_s1[group_idx][elem_idx] -= np.pi / 2

            # Overlap for shifted s1
            overlap_plus = calculate_overlap(qnode_func, params_plus_s1, params)
            overlap_minus = calculate_overlap(qnode_func, params_minus_s1, params)

            # Gradient contribution from S1
            grad_s1 = (loss_function(overlap_plus, is_duplicate) - loss_function(overlap_minus, is_duplicate)) / 2

            # Now, shift the parameters for the second sentence (S2)
            params_plus_s2 = params.copy()
            params_plus_s2[group_idx][elem_idx] += np.pi / 2
            
            params_minus_s2 = params.copy()
            params_minus_s2[group_idx][elem_idx] -= np.pi / 2

            # Overlap for shifted S2
            overlap_plus_2 = calculate_overlap(qnode_func, params, params_plus_s2)
            overlap_minus_2 = calculate_overlap(qnode_func, params, params_minus_s2)

            # Gradient contribution from S2
            grad_s2 = (loss_function(overlap_plus_2, is_duplicate) - loss_function(overlap_minus_2, is_duplicate)) / 2
            
            # The total gradient is the sum of contributions from both states
            gradients[param_index] = grad_s1 + grad_s2

    return gradients

def group_sentences_by_qubit_count(sentences, ansatz, parser, rewriter):
    """
    Groups sentences based on the number of qubits in their corresponding circuit.
    
    Returns a dictionary where keys are qubit counts and values are lists of sentences.
    """
    print("Grouping sentences by qubit count...")
    grouped_sentences = {}
    for sentence in sentences:
        try:
            diagram = parser.sentence2diagram(sentence)
            rewritten_diagram = rewriter(diagram)
            normalized_diagram = rewritten_diagram.normal_form()
            circuit = ansatz(normalized_diagram)

            # --- CORRECTION: Get the number of qubits from the circuit object directly ---
            temp_qml_circuit = to_qml(circuit)
            num_qubits = temp_qml_circuit._n_qubits
            
            if num_qubits not in grouped_sentences:
                grouped_sentences[num_qubits] = []
                
            grouped_sentences[num_qubits].append(sentence)
            
        except Exception as e:
            print(f"Skipping sentence '{sentence}' due to an error: {e}")

    print("Grouping complete.")
    for n_qubits, s_list in grouped_sentences.items():
        print(f"  - {n_qubits} qubits: {len(s_list)} sentences")
        
    return grouped_sentences

# Assuming all other helper functions are defined as per previous responses

def main_workflow(data_pairs, parser, rewriter, ansatz):
    """
    The main function to perform per-dimension training based on sentence pairs.
    
    Args:
        data_pairs (list): A list of tuples, each containing (sentence1, sentence2, is_duplicate).
        parser (lambeq.parser.Parser): The parser.
        rewriter (lambeq.rewriter.Rewriter): The rewriter.
        ansatz (lambeq.ansatz.Ansatz): The quantum ansatz.
    """
    print("Grouping sentence pairs by qubit count...")
    grouped_pairs = {}
    
    for s1, s2, is_duplicate in data_pairs:
        try:
            _, _, num_qubits_s1 = lambeq_sentence_to_circuit(s1, ansatz, parser, rewriter, return_type='state')
            _, _, num_qubits_s2 = lambeq_sentence_to_circuit(s2, ansatz, parser, rewriter, return_type='state')
            
            if num_qubits_s1 is None or num_qubits_s2 is None or num_qubits_s1 != num_qubits_s2:
                print(f"Skipping pair ('{s1}', '{s2}') due to different or invalid qubit counts.")
                continue

            if num_qubits_s1 not in grouped_pairs:
                grouped_pairs[num_qubits_s1] = []

            grouped_pairs[num_qubits_s1].append((s1, s2, is_duplicate))
            
        except Exception as e:
            print(f"Skipping pair due to an error: {e}")

    print("Grouping complete.")
    for n_qubits, s_list in grouped_pairs.items():
        print(f"  - {n_qubits} qubits: {len(s_list)} sentence pairs")
        
    trained_params = {}
    print("\n--- Starting Training ---")
    
    for num_qubits, pair_list in grouped_pairs.items():
        if not pair_list:
            continue
            
        print(f"\nTraining model for {num_qubits} qubits with {len(pair_list)} pairs...")
        
        # Get the blueprint for the circuit and parameters
        rep_sentence = pair_list[0][0]
        qnode_func, param_structure, _ = lambeq_sentence_to_circuit(
            rep_sentence, ansatz, parser, rewriter, return_type='expval', expval=qml.PauliZ(0)
        )
        
        # Initialize a single set of parameters for this dimension
        params = initialize_structured_params(param_structure)
        
        # --- Training loop over pairs ---
        learning_rate = 0.01
        for s1, s2, is_duplicate in pair_list:
            
            gradients = calculate_pair_gradients(qnode_func, params, s1, s2, is_duplicate)
            
            for group_idx, group in enumerate(params):
                for elem_idx in range(len(group)):
                    param_index = (group_idx, elem_idx)
                    params[group_idx][elem_idx] -= learning_rate * gradients[param_index]
            
        trained_params[num_qubits] = params
    
    print("\n--- Training Complete ---")
    
    print("\n--- Performing Final Swap Tests with Trained Parameters ---")
    for num_qubits, pair_list in grouped_pairs.items():
        if not pair_list:
            continue
        
        sample_pairs = pair_list[:5] if len(pair_list) > 5 else pair_list
        
        for s1, s2, is_duplicate in sample_pairs:
            print(f"\nEvaluating Pair: '{s1}' vs. '{s2}' (Label: {'Duplicate' if is_duplicate else 'Not Duplicate'})")
            
            optimized_params = trained_params[num_qubits]
            
            qnode_for_state, _, _ = lambeq_sentence_to_circuit(
                s1, ansatz, parser, rewriter, return_type='state'
            )
            vec1 = qnode_for_state(optimized_params)
            
            qnode_for_state_2, _, _ = lambeq_sentence_to_circuit(
                s2, ansatz, parser, rewriter, return_type='state'
            )
            vec2 = qnode_for_state_2(optimized_params)

            overlap = swap_test(vec1, vec2, num_qubits)
            print(f"Final Overlap: {overlap:.4f}")
            Qangle = FischerInformation(overlap)
            NormalizedQangle = Qangle / (math.pi / 2)
            print(f"Estimated Quantum Angle: {NormalizedQangle:.4f}")

In [55]:
main_workflow(data_pairs, parser, rewriter, ansatz)

Grouping sentence pairs by qubit count...
Skipping pair ('How does the Boggart work?', 'What would the boggart of a boggart be?') due to different or invalid qubit counts.
CRITICAL ERROR: Failed to process circuit: Bobcat failed to parse 'What hotel in Jabalpur would be safe for unmarried couples, without the harassment of police, hotel staff, and moral police?'.
Skipping pair due to an error: not enough values to unpack (expected 3, got 2)


Traceback (most recent call last):
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\text2diagram\model_based_reader\bobcat_parser.py", line 265, in sentences2trees
    trees.append(self._build_ccgtree(result[0]))
                                     ~~~~~~^^^
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\bobcat\parser.py", line 256, in __getitem__
    return self.root[index]
           ~~~~~~~~~^^^^^^^
IndexError: list index out of range

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Jash\AppData\Local\Temp\ipykernel_9820\2306687431.py", line 27, in lambeq_sentence_to_circuit
    diagram = parser.sentence2diagram(sentence)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\text2diagram\ccg_parser.py", line 248, in sentence2diagram
  

CRITICAL ERROR: Failed to process circuit: Ty(p)
Skipping pair due to an error: not enough values to unpack (expected 3, got 2)


Traceback (most recent call last):
  File "C:\Users\Jash\AppData\Local\Temp\ipykernel_9820\2306687431.py", line 42, in lambeq_sentence_to_circuit
    circuit = ansatz(normalized_diagram)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\ansatz\circuit.py", line 120, in __call__
    return self.functor(diagram)  # type: ignore[return-value]
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\backend\grammar.py", line 2046, in __call__
    return self.ar_with_cache(entity)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\backend\grammar.py", line 2077, in ar_with_cache
    ret = ar.apply_functor(self)
          ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\backend\grammar.py", line 1

Skipping pair ('How do I fill in Address Line 1 and Address Line 2?', 'How do I register desired web address?') due to different or invalid qubit counts.
Skipping pair ('What is the procedure to join the Bomb Detection and Disposal Squad after metallurgy engineering?', 'Where can we get medical reports in Delhi?') due to different or invalid qubit counts.
CRITICAL ERROR: Failed to process circuit: Ty(p)


...

Skipping pair due to an error: not enough values to unpack (expected 3, got 2)
Skipping pair ('What does it mean when you see no icons on sent Facebook messege?', 'What does it mean when a person wishes you happy birthday in a Facebook message not on your timeline?') due to different or invalid qubit counts.
Skipping pair ('How do GST bill will benefit local people?', 'What exactly is GST bill and how exactly will it affect the common people?') due to different or invalid qubit counts.
Skipping pair ('Why can't India get more medals in Olympics?', 'Why can't India won Olympic gold medals despite the fact that African countries are winning more medal than we are?') due to different or invalid qubit counts.
CRITICAL ERROR: Failed to process circuit: Bobcat failed to parse 'What is the best way to save/ invest money?'.
Skipping pair due to an error: not enough values to unpack (expected 3, got 2)


Traceback (most recent call last):
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\text2diagram\model_based_reader\bobcat_parser.py", line 265, in sentences2trees
    trees.append(self._build_ccgtree(result[0]))
                                     ~~~~~~^^^
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\bobcat\parser.py", line 256, in __getitem__
    return self.root[index]
           ~~~~~~~~~^^^^^^^
IndexError: list index out of range

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Jash\AppData\Local\Temp\ipykernel_9820\2306687431.py", line 27, in lambeq_sentence_to_circuit
    diagram = parser.sentence2diagram(sentence)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\text2diagram\ccg_parser.py", line 248, in sentence2diagram
  

CRITICAL ERROR: Failed to process circuit: Ty(p)
Skipping pair due to an error: not enough values to unpack (expected 3, got 2)


Traceback (most recent call last):
  File "C:\Users\Jash\AppData\Local\Temp\ipykernel_9820\2306687431.py", line 42, in lambeq_sentence_to_circuit
    circuit = ansatz(normalized_diagram)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\ansatz\circuit.py", line 120, in __call__
    return self.functor(diagram)  # type: ignore[return-value]
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\backend\grammar.py", line 2046, in __call__
    return self.ar_with_cache(entity)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\backend\grammar.py", line 2077, in ar_with_cache
    ret = ar.apply_functor(self)
          ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jash\Documents\Research\Semantic Equivilance\.venv\Lib\site-packages\lambeq\backend\grammar.py", line 1

KeyboardInterrupt: 